In [1]:
! pip install git+git://github.com/frehseg/nvcc4jupyter.git

  Cloning git://github.com/frehseg/nvcc4jupyter.git to /tmp/pip-req-build-r7xnfk_e
  Running command git clone -q git://github.com/frehseg/nvcc4jupyter.git /tmp/pip-req-build-r7xnfk_e
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.1-cp36-none-any.whl size=2095 sha256=b88320ba66674a21bc8669178806597ac893cae8fc32a9ed33d5c6587cdf82e1
  Stored in directory: /tmp/pip-ephem-wheel-cache-z_fdg6pl/wheels/a4/a5/24/17a2b61f9a725a10155cc6fca753aae28436921df21fa16114
Successfully built NVCCPlugin


In [0]:
%load_ext nvcc_plugin

### Version basique

Version n'utilisant que la mémoire globale du GPU. Chaque bloc étant limité à 1024 threads, une tuile carrée a des dimensions maximales de 32x32.

In [3]:
%%cu
#include <stdlib.h>
#include <stdio.h>

#include <time.h>

#include <cuda.h>
#include <cuda_runtime.h>
#include <helper_cuda.h>

#define TILE_WIDTH 16
#define SIZE 20

/********************** kernel **************************/
__global__
void matmul(float *A, float *B, float *C, int nb_ColA, int nb_ColB, int nb_LigneA, int nb_LigneB)
{
  int ligne = blockIdx.y * blockDim.y + threadIdx.y;
  int col = blockIdx.x * blockDim.x + threadIdx.x;
  if (ligne < nb_LigneA && col < nb_LigneB) {
      for(int i = 0; i < nb_ColA; i++){
          C[ligne * nb_ColB + col] += A[ligne * nb_ColA + i] * B[i * nb_ColB + col];
      }
  }
}

/********************** main **************************/
int main(void)
{
  int dev = 0;
  cudaDeviceProp deviceProp;
  cudaGetDeviceProperties(&deviceProp, dev);
  printf("device %d: %s ", dev, deviceProp.name);
  cudaSetDevice(dev);
 
  int n_iter = 10;
  for(int j = 0; j < n_iter; j++){
      
  float *A, *B, *C, *gpu_A, *gpu_B, *gpu_C;
  int nbLigneA, nbLigneB, nbColA, nbColB;

  
  nbLigneA = TILE_WIDTH * SIZE;
  nbLigneB = TILE_WIDTH * SIZE;
  nbColA = TILE_WIDTH * SIZE;
  nbColB = TILE_WIDTH * SIZE;

  A = (float*) malloc(nbLigneA * nbColA * sizeof(float));
  B = (float*) malloc(nbLigneB * nbColB * sizeof(float));
  C = (float*) malloc(nbLigneA * nbColB * sizeof(float));

  /*Allocation de l'espace pour le GPU */
  cudaMalloc((void**) &gpu_A, nbLigneA * nbColA * sizeof(float));
  cudaMalloc((void**) &gpu_B, nbLigneB * nbColB * sizeof(float));
  cudaMalloc((void**) &gpu_C, nbLigneA * nbColB * sizeof(float));

  /* Initialisation de A et B*/
  for (int i = 0; i < nbLigneA * nbColA; i++) {
    A[i] = 1.0;
  }

  for (int i = 0; i < nbLigneB * nbColB; i++) {
    B[i] = 2.0;
  }

  /* Copie de A et B sur le GPU */
  cudaMemcpy(gpu_A, A, nbLigneA * nbColA * sizeof(float), cudaMemcpyHostToDevice);
  cudaMemcpy(gpu_B, B, nbLigneB * nbColB * sizeof(float), cudaMemcpyHostToDevice);
  cudaMemcpy(gpu_C, C, nbLigneA * nbColB * sizeof(float), cudaMemcpyHostToDevice);


  /* Lancement du kernel avec mesure du temps */
 dim3 threadsPerBlock (TILE_WIDTH, TILE_WIDTH);
 dim3 blocksPerGrid (nbLigneA / threadsPerBlock.y, nbLigneB / threadsPerBlock.x);

 clock_t start = clock();

 matmul<<<blocksPerGrid, threadsPerBlock>>>(gpu_A, gpu_B, gpu_C, nbColA, nbColB, nbLigneA, nbLigneB);

 clock_t end = clock();
 if(j == 0){

  cudaMemcpy(C, gpu_C, nbLigneA * nbColB * sizeof(float), cudaMemcpyDeviceToHost);

  /* Vérification du résultat*/
  float maxError = 0.0f;
  for (int i = 0; i < nbLigneA * nbColB; i++){
      maxError = max(maxError, abs(C[i]- 2*nbLigneB));
  }
  printf("Max error: %f\n", maxError);
 }

 double time_taken = ((double) end-start)/CLOCKS_PER_SEC; // in seconds
 printf("\n Kernel took %f seconds to execute\n\n", time_taken);

  /* Libération de la mémoire sur le GPU*/ 
  cudaFree(gpu_A);
  cudaFree(gpu_B);
  cudaFree(gpu_C);

  free(A);
  free(B);
  free(C);
}
}

device 0: Tesla P4 Max error: 0.000000

 Kernel took 0.000017 seconds to execute


 Kernel took 0.000013 seconds to execute


 Kernel took 0.000012 seconds to execute


 Kernel took 0.000011 seconds to execute


 Kernel took 0.000011 seconds to execute


 Kernel took 0.000011 seconds to execute


 Kernel took 0.000011 seconds to execute


 Kernel took 0.000011 seconds to execute


 Kernel took 0.000011 seconds to execute


 Kernel took 0.000011 seconds to execute




#### Pour la version basique, le temps d'execution du kernel semble se stabiliser entre 1e-5 et 2.5e-5 sec. Using TILE_WIDTH of 32 or 16 doesn't impact the results.

#### Regardons maintenant l'influence de l'utilisation d'un GPU en comparant le resultat avec la version CPU:


In [4]:
%%cu
#include <stdlib.h>
#include <stdio.h>

#include <time.h>

#include <cuda.h>
#include <cuda_runtime.h>
#include <helper_cuda.h>

#define TILE_WIDTH 16
#define SIZE 20

/********************** kernel **************************/
void matmul(float *A, float *B, float *C, int nb_ColA, int nb_ColB, int nb_LigneA, int nb_LigneB)
{
  for (int i = 0; i < nb_LigneA; i++) {
        for (int j = 0; j < nb_ColB; j++) {
            int sum = 0;
            for (int k = 0; k < nb_ColA; k++)
                sum = sum + A[i * nb_ColA + k] * B[k * nb_ColB + j];
            C[i * nb_ColB + j] = sum;
        }
    }
}

/********************** main **************************/
int main(void)
{
  int dev = 0;
  cudaDeviceProp deviceProp;
  cudaGetDeviceProperties(&deviceProp, dev);
  printf("device %d: %s ", dev, deviceProp.name);
  cudaSetDevice(dev);

  int n_iter = 10;
  for(int j = 0; j < n_iter; j++){
      
  float *A, *B, *C;
  int nbLigneA, nbLigneB, nbColA, nbColB;

  nbLigneA = TILE_WIDTH * SIZE;
  nbLigneB = TILE_WIDTH * SIZE;
  nbColA = TILE_WIDTH * SIZE;
  nbColB = TILE_WIDTH * SIZE;

  A = (float*) malloc(nbLigneA * nbColA * sizeof(float));
  B = (float*) malloc(nbLigneB * nbColB * sizeof(float));
  C = (float*) malloc(nbLigneA * nbColB * sizeof(float));

  /* Initialisation de A et B*/
  for (int i = 0; i < nbLigneA * nbColA; i++) {
    A[i] = 1.0;
  }

  for (int i = 0; i < nbLigneB * nbColB; i++) {
    B[i] = 2.0;
  }

  /* Lancement du kernel avec mesure du temps */

 clock_t start = clock();

 matmul(A, B, C, nbColA, nbColB, nbLigneA, nbLigneB);

 clock_t end = clock();

 if(j == 0){

  /* Vérification du résultat*/
  float maxError = 0.0f;
  for (int i = 0; i < nbLigneA * nbColB; i++){
      maxError = max(maxError, abs(C[i]- 2*nbLigneB));
  }
  printf("Max error: %f\n", maxError);
 }

 double time_taken = ((double) end-start)/CLOCKS_PER_SEC; // in seconds
 printf("\n Kernel took %f seconds to execute\n\n", time_taken);

  free(A);
  free(B);
  free(C);
}
}

device 0: Tesla P4 Max error: 0.000000

 Kernel took 0.261367 seconds to execute


 Kernel took 0.258055 seconds to execute


 Kernel took 0.208877 seconds to execute


 Kernel took 0.209259 seconds to execute


 Kernel took 0.210120 seconds to execute


 Kernel took 0.209867 seconds to execute


 Kernel took 0.214595 seconds to execute


 Kernel took 0.222661 seconds to execute


 Kernel took 0.208501 seconds to execute


 Kernel took 0.210239 seconds to execute




#### La version CPU mets entre 1.7 et 1.9 sec pour s'executer avec un TILE_WIDTH de 32. Alors qu'avec un TILE_WIDTH de 16, le programme met entre 0.2 et 0.26 sec.

### Version tuilée:

Les algorithmes de produit de matrices les plus performants décomposent les algorithmes en tuile. 

- Chaque thread de chaque bloc calcule un élément la matrice C.
- Dans chaque bloc, une tuile de la matrice A et une tuile de la matrice B sont stockées en mémoire partagée (dans des vecteurs 2 dimensions [TILE_WIDTH][TILE_WIDTH], stockés sous la forme de tableaux uni-dimensionnels row-major).
-  Chaque thread copie l'élément des matrices A et B correspondant à ses coordonnées en mémoire partagée, attend que les données sont disponibles puis effectue le calcul et le stocke dans une variable temporaire.
- Tant que le calcul n'est pas fini, le thread met à jour cette même variable temporaire.
- La valeur temporaire est recopiée en mémoire globale dans la matrice C.

#### Comparaison de la version CPU, basique et de la version tuilee:

In [39]:
%%cu
#include <stdlib.h>
#include <stdio.h>

#include <time.h>

#include <cuda.h>
#include <cuda_runtime.h>
#include <helper_cuda.h>
#include <cublas_v2.h>

#define TILE_WIDTH 16
#define SIZE 20


/* CPU Version */
void matmul(float *A, float *B, float *C, int nb_ColA, int nb_ColB, int nb_LigneA, int nb_LigneB)
{
  for (int i = 0; i < nb_LigneA; i++) {
        for (int j = 0; j < nb_ColB; j++) {
            int sum = 0;
            for (int k = 0; k < nb_ColA; k++)
                sum = sum + A[i * nb_ColA + k] * B[k * nb_ColB + j];
            C[i * nb_ColB + j] = sum;
        }
    }
}

/* Version Basic */
__global__
void basicmatmul(float *A, float *B, float *C, int nb_ColA, int nb_ColB, int nb_LigneA, int nb_LigneB)
{
  int ligne = blockIdx.y * blockDim.y + threadIdx.y;
  int col = blockIdx.x * blockDim.x + threadIdx.x;
  if (ligne < nb_LigneA && col < nb_LigneB) {
      for(int i = 0; i < nb_ColA; i++){
          C[ligne * nb_ColB + col] += A[ligne * nb_ColA + i] * B[i * nb_ColB + col];
      }
  }
}

/* Version Tuilee */
__global__
void tuileematmul(float *A, float *B, float *C, int nb_ColA, int nb_ColB, int nb_LigneA, int nb_LigneB)
{
    int blockRow = blockIdx.y;
    int blockCol = blockIdx.x;
    float* Csub = &C[nb_ColB * TILE_WIDTH * blockRow + TILE_WIDTH * blockCol];
    float Cvalue = 0;
    int row = threadIdx.y;
    int col = threadIdx.x;
    for (int m = 0; m < (nb_ColA / TILE_WIDTH); ++m) {
        float* Asub = &A[nb_ColA * TILE_WIDTH * blockRow + TILE_WIDTH * m];
        float* Bsub = &B[nb_ColB * TILE_WIDTH * m + TILE_WIDTH * blockCol];
        __shared__ float As[TILE_WIDTH][TILE_WIDTH];
        __shared__ float Bs[TILE_WIDTH][TILE_WIDTH];
        As[row][col] = Asub[row * nb_ColA + col];
        Bs[row][col] = Bsub[row * nb_ColB + col];
        __syncthreads();
        for (int e = 0; e < TILE_WIDTH; ++e)
            Cvalue += As[row][e] * Bs[e][col];
        __syncthreads();
    }
    Csub[row * nb_ColA + col] = Cvalue;
}

/********************** main **************************/
int main(void)
{
  int dev = 0;
  cudaDeviceProp deviceProp;
  cudaGetDeviceProperties(&deviceProp, dev);
  printf("device %d: %s ", dev, deviceProp.name);
  cudaSetDevice(dev);

  float *A, *B, *C, *gpu_A, *gpu_B, *gpu_C;
  int nbRowA, nbRowB, nbColA, nbColB;

  
  nbRowA = TILE_WIDTH * SIZE;
  nbRowB = TILE_WIDTH * SIZE;
  nbColA = TILE_WIDTH * SIZE;
  nbColB = TILE_WIDTH * SIZE;

  A = (float*) malloc(nbRowA * nbColA * sizeof(float));
  B = (float*) malloc(nbRowB * nbColB * sizeof(float));
  C = (float*) malloc(nbRowA * nbColB * sizeof(float));

  /* GPU space allocation */
  cudaMalloc((void**) &gpu_A, nbRowA * nbColA * sizeof(float));
  cudaMalloc((void**) &gpu_B, nbRowB * nbColB * sizeof(float));
  cudaMalloc((void**) &gpu_C, nbRowA * nbColB * sizeof(float));

  /* A & B initialization */
  for (int i = 0; i < nbRowA * nbColA; i++) {
    A[i] = 1.0;
  }

  for (int i = 0; i < nbRowB * nbColB; i++) {
    B[i] = 2.0;
  }

  /* Copy of A & B on GPU */
  cudaMemcpy(gpu_A, A, nbRowA * nbColA * sizeof(float), cudaMemcpyHostToDevice);
  cudaMemcpy(gpu_B, B, nbRowB * nbColB * sizeof(float), cudaMemcpyHostToDevice);
  cudaMemcpy(gpu_C, C, nbRowA * nbColB * sizeof(float), cudaMemcpyHostToDevice);


 dim3 threadsPerBlock (TILE_WIDTH, TILE_WIDTH);
 dim3 blocksPerGrid (nbRowA / threadsPerBlock.y, nbRowB / threadsPerBlock.x);

 double cpu_time = 0.0;
 float cpu_max_error = 0.0f;
 double basic_time = 0.0;
 float basic_max_error = 0.0f;
 double shared_mem_time = 0.0;
 float shared_mem_max_error = 0.0f;

 int n_iter = 20;
 for(int i = 0; i < n_iter; i++){
     
 /* CPU */
 clock_t start = clock();
 matmul(A, B, C, nbColA, nbColB, nbRowA, nbRowB);
 clock_t end = clock();
 cpu_time += ((double) end-start)/CLOCKS_PER_SEC; // in seconds
 if(i == 0){
    /* Vérification du résultat*/
    for (int i = 0; i < nbRowA * nbColB; i++){
        cpu_max_error = max(cpu_max_error, abs(C[i]- 2*nbRowB));
    }
  }

 /* Basic Version */
 start = clock();
 basicmatmul<<<blocksPerGrid, threadsPerBlock>>>(gpu_A, gpu_B, gpu_C, nbColA, nbColB, nbRowA, nbRowB);
 end = clock();
 basic_time += ((double) end-start)/CLOCKS_PER_SEC; // in seconds
 if(i == 0){
    cudaMemcpy(C, gpu_C, nbRowA * nbColB * sizeof(float), cudaMemcpyDeviceToHost);
    /* Vérification du résultat*/
    for (int i = 0; i < nbRowA * nbColB; i++){
        basic_max_error = max(basic_max_error, abs(C[i]- 2*nbRowB));
    }
  }

 /* Shared Memory Version */
 start = clock();
 tuileematmul<<<blocksPerGrid, threadsPerBlock>>>(gpu_A, gpu_B, gpu_C, nbColA, nbColB, nbRowA, nbRowB);
 end = clock();
 shared_mem_time += ((double) end-start)/CLOCKS_PER_SEC; // in seconds
 if(i == 0){
    cudaMemcpy(C, gpu_C, nbRowA * nbColB * sizeof(float), cudaMemcpyDeviceToHost);
    /* Vérification du résultat*/
    for (int i = 0; i < nbRowA * nbColB; i++){
        shared_mem_max_error = max(shared_mem_max_error, abs(C[i]- 2*nbRowB));
    }
  }
 }
 
 printf("\nCPU version took %f seconds to execute \n", cpu_time/n_iter);
 printf("Max error: %f\n", cpu_max_error);
 
 printf("\nBasic version took %f seconds to execute \n", basic_time/n_iter);
 printf("Max error: %f\n", basic_max_error);

 printf("\nShared memory version took %f seconds to execute \n", shared_mem_time/n_iter);
 printf("Max error: %f\n", shared_mem_max_error);

  /* Libération de la mémoire sur le GPU*/ 
  cudaFree(gpu_A);
  cudaFree(gpu_B);
  cudaFree(gpu_C);

  free(A);
  free(B);
  free(C);
}

device 0: Tesla P4 
CPU version took 0.212490 seconds to execute 
Max error: 0.000000

Basic version took 0.000028 seconds to execute 
Max error: 0.000000

Shared memory version took 0.000006 seconds to execute 
Max error: 0.000000



#### CuBLAS version


In [38]:
%%cu
#include <stdlib.h>
#include <stdio.h>

#include <time.h>

#include <cuda.h>
#include <cuda_runtime.h>
#include <helper_cuda.h>
#include <cublas_v2.h>

#define TILE_WIDTH 32
#define SIZE 20

/********************** main **************************/
int main(void)
{
  int dev = 0;
  cudaDeviceProp deviceProp;
  cudaGetDeviceProperties(&deviceProp, dev);
  printf("device %d: %s ", dev, deviceProp.name);
  cudaSetDevice(dev);
 
  float *A, *B, *C, *gpu_A, *gpu_B, *gpu_C;
  int nbRowA, nbRowB, nbColA, nbColB;

  
  nbRowA = TILE_WIDTH * SIZE;
  nbRowB = TILE_WIDTH * SIZE;
  nbColA = TILE_WIDTH * SIZE;
  nbColB = TILE_WIDTH * SIZE;

  A = (float*) malloc(nbRowA * nbColA * sizeof(float));
  B = (float*) malloc(nbRowB * nbColB * sizeof(float));
  C = (float*) malloc(nbRowA * nbColB * sizeof(float));

  /* GPU space allocation */
  cudaMalloc((void**) &gpu_A, nbRowA * nbColA * sizeof(float));
  cudaMalloc((void**) &gpu_B, nbRowB * nbColB * sizeof(float));
  cudaMalloc((void**) &gpu_C, nbRowA * nbColB * sizeof(float));

  /* A & B initialization */
  for (int i = 0; i < nbRowA * nbColA; i++) {
    A[i] = 1.0;
  }

  for (int i = 0; i < nbRowB * nbColB; i++) {
    B[i] = 2.0;
  }

  /* Copy of A & B on GPU */
  cudaMemcpy(gpu_A, A, nbRowA * nbColA * sizeof(float), cudaMemcpyHostToDevice);
  cudaMemcpy(gpu_B, B, nbRowB * nbColB * sizeof(float), cudaMemcpyHostToDevice);
  cudaMemcpy(gpu_C, C, nbRowA * nbColB * sizeof(float), cudaMemcpyHostToDevice);


 dim3 threadsPerBlock (TILE_WIDTH, TILE_WIDTH);
 dim3 blocksPerGrid (nbRowA / threadsPerBlock.y, nbRowB / threadsPerBlock.x);

 double cuBLAS_time = 0.0;
 float cuBLAS_max_error = 0.0f;

int n_iter = 20;
 for(int i = 0; i < n_iter; i++){


 /* cuBLAS Version */
 const float alf = 1;
 const float bet = 0;
 const float *alpha = &alf;
 const float *beta = &bet;
 
 cublasHandle_t handle;
 cublasCreate(&handle);
 
 /* Warming up */
 cublasSgemm(handle, CUBLAS_OP_N, CUBLAS_OP_N, nbRowA, nbColB, nbColA, alpha, gpu_A, nbRowA, gpu_B, nbColA, beta, gpu_C, nbRowA);

 clock_t start = clock();
 cublasSgemm(handle, CUBLAS_OP_N, CUBLAS_OP_N, nbRowA, nbColB, nbColA, alpha, gpu_A, nbRowA, gpu_B, nbColA, beta, gpu_C, nbRowA);
 clock_t end = clock();

 // Destroy the handle
 cublasDestroy(handle);

 cuBLAS_time += ((double) end-start)/CLOCKS_PER_SEC; // in seconds
 if(i == 0){
    cudaMemcpy(C, gpu_C, nbRowA * nbColB * sizeof(float), cudaMemcpyDeviceToHost);
    /* Vérification du résultat*/
    for (int i = 0; i < nbRowA * nbColB; i++){
        cuBLAS_max_error = max(cuBLAS_max_error, abs(C[i]- 2*nbRowB));
    }
  }
 }
 printf("\ncuBLAS version took %f seconds to execute \n", cuBLAS_time/n_iter);
 printf("Max error: %f\n", cuBLAS_max_error);

  /* Libération de la mémoire sur le GPU*/ 
  cudaFree(gpu_A);
  cudaFree(gpu_B);
  cudaFree(gpu_C);

  free(A);
  free(B);
  free(C);
 
}


device 0: Tesla P4 
cuBLAS version took 0.000008 seconds to execute 
Max error: 0.000000

